In [ ]:
!pip install simpletransformers

In [ ]:
import os, collections
import numpy as np

def read_dataBert(path):
    sentences = []
    words = []
    labels = []
    complex_word_index = []
    file = open(path)
    lines = file.readlines()
    previousSentence = None
    currentWords = []
    currentIndex = []
    currentLabels= []
    for line in lines : 
        line_split = line.split('\t')
        currentSentence = line_split[0]
        if not currentSentence == previousSentence:
          sentences.append(currentSentence)
          previousSentence = currentSentence
          words.append(currentWords)
          currentWords = []
          complex_word_index.append(currentIndex)
          currentIndex = []
          labels.append(currentLabels)
          currentLabels = []

        currentWords.append(line_split[1]) # word
        cwi_index = int(line_split[2])
        currentIndex.append(cwi_index) # word index needed for context
        ##### pour etre entre 0 et 1
        currentLabels.append(int(line_split[3][:-1])) # label
        ##### pour etre entre -1 et 1
        #currentLabels.append(int(line_split[3][:-1])*2-1) # label
    words.pop(0); labels.pop(0); complex_word_index.pop(0)
    return sentences, words, labels, complex_word_index

def words_to_index(sentences, words):
    vocab = collections.defaultdict(lambda: len(vocab))
    vocab['<eos>'] = 0
    
    int_sentences = []
    int_words = []
    for i, sentence  in enumerate(sentences):
        sentence_words = sentence.split()
        int_sentences.append([vocab[token.lower()] for token in sentence_words])
        int_words.append(vocab[words[i].lower()])
            
    return int_sentences, int_words

def word_context(sentences, indecies, context_size=2):
    pre_context, post_context = [], []
    for i, sentence in enumerate(sentences):
        #pre_word_context, post_word_context = [], []
        word_index = indecies[i]
        upper_bound = word_index + context_size + 1
        lower_bound = word_index - context_size - 1
        if lower_bound <= 0:
            lower_bound = None
        pre_context.append(sentence.split()[word_index-1 : lower_bound: -1][::-1])
        post_context.append(sentence.split()[word_index+1 : upper_bound])
        
    return pre_context, post_context

def tokenize(sentences):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    tokenized = []
    for context in sentences:
        tokens = []
        for word in context:
            tokenised_word = tokenizer.encode(word, add_special_tokens=True)
            tokens += tokenised_word
        tokenized.append(tokens)
    return tokenized

def build_attention_map(pre_tokens, word_tokens, max_size):
    # pre_tokens, word_tokens are lists for each word ...
    attention_maps = []
    for i in range(len(pre_tokens)):
        attention_map = [0]*max_size
        max = len(pre_tokens[i])+len(word_tokens[i])
        attention_map[len(pre_tokens[i]): max] = [1]*len(word_tokens[i])
        attention_maps.append(attention_maps)
            
    return attention_maps



In [ ]:

import os
from google.colab import drive
drive.mount('/content/drive')

path = 'drive/MyDrive/pstaln'

train_data_file = path+'/data/cwi_training/cwi_training.txt'
test_data_file = path+'/data/cwi_testing_annotated/cwi_testing_annotated.txt'
output_file =path+ '/output/test.txt'

train_sentences, train_words, train_label, train_ind = read_dataBert(train_data_file)
test_sentences, test_words, test_label, test_ind = read_dataBert(test_data_file)

print(train_sentences[-1])
print('words',train_words[1])
print('labels',train_label[1])
print('index',train_ind[1])
#test_sentences, test_words, test_label, x2 = read_data(test_data_file)


In [ ]:
from simpletransformers.language_representation import RepresentationModel

model = RepresentationModel(
        model_type="bert",
        model_name="bert-base-uncased",
        use_cuda=False
    )

sentence_vectors = model.encode_sentences(train_sentences, combine_strategy=None,batch_size=len(train_sentences))

In [ ]:
print(len(test_sentences), len(test_ind), len(test_label))

In [ ]:
def getWordVector2(sentence_vectors, index, labels):
  x = []
  y = []
  for sentence in range(len(index)):

    for word in index[sentence]:

      x.append(sentence_vectors[sentence][word+1])
  for k in labels:
    for i in k:
      yNew = -1
      if i == 1:
        yNew=1 
      y.append(yNew)
  return x, y 

def getWordVector(sentence_vectors, index, labels):
  x = []
  y = []
  for sentence in range(len(index)):

    for word in index[sentence]:

      x.append(sentence_vectors[sentence][word+1])
  for k in labels:
    for i in k:
      yNew = np.zeros(2)
      if i == 1:
        yNew[1]=1
      else:
        yNew[0]=1 
      y.append(yNew)
  return x, y 
x, y = getWordVector(sentence_vectors, train_ind,train_label)
print(len(x), len(y))

In [ ]:
import tensorflow as tf
classifier = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation = 'relu'),
    tf.keras.layers.Dense(2)])
'''
import tensorflow as tf
classifier = tf.keras.Sequential([
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(2)])
'''
classifier.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
classifier.fit(np.array(x), np.array(y), epochs=10)


In [ ]:
'''
import tensorflow as tf
classifier = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation = 'relu'),
    tf.keras.layers.Dense(2)])

classifier.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

classifier.fit(np.array(x), np.array(y), epochs=10)
'''
def convertToArray(lst):
  for k in range(len(lst)):
    lst[k] = np.array(lst[k])
  return np.array(lst)
def split(a, n):
    k, m = divmod(len(a), n)
    return np.array([a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)])
def predict(sentences, index, classifier=classifier, embeddingModel = model):
  a = len(sentences)
  numberSplit = a // 100
  sntc = split(sentences, numberSplit)
  id = split(index, numberSplit)
  pred = []
  for k in range(len(sntc)):
    print(k,' // ', numberSplit)
    p = predictBatch(sntc[k], id[k],  classifier, embeddingModel)
    pred.append(p)
  return(pred)
def predictBatch(sentences, index,classifier=classifier, embeddingModel = model):
  sentence_vectors = model.encode_sentences(sentences, combine_strategy=None,batch_size=len(sentences))
  x,y = getWordVector(sentence_vectors, index,[[1]])
  x = convertToArray(x)
  predictions = classifier.predict(x)
  return(predictions)
def evaluate(sentences, index,labels, classifier=classifier, embeddingModel = model):
  a = len(sentences)
  numberSplit = a // 100
  sntc = split(sentences, numberSplit)
  id = split(index, numberSplit)
  lab = split(labels, numberSplit)
  test_loss, test_acc = 0,0
  for k in range(len(sntc)):
    print(k,' // ', numberSplit)
    l,a = evaluateBatch(sntc[k], id[k], lab[k], classifier, embeddingModel)
    test_loss+=l; test_acc+=a
  print('loss is : ', test_loss/numberSplit,' and acc is : ', test_acc/numberSplit)

def evaluateBatch(sentences, index,labels, classifier=classifier, embeddingModel = model):
  #print(len(sentences), len(index), len(labels))
  sentence_vectors = model.encode_sentences(sentences, combine_strategy=None,batch_size=len(sentences))
  x,y = getWordVector(sentence_vectors, index,labels)
  x = convertToArray(x)
  #print(x.shape)
  y = convertToArray(y)
  #print(y.shape)
  test_loss, test_acc = classifier.evaluate(x,y, verbose=0)
  return(test_loss, test_acc)



In [ ]:
####Decommenter pour voir les resultats sur le test

evaluate(test_sentences, test_ind, test_label)

In [ ]:
####Decommenter pour voir la recall sur le test

pred = predict(test_sentences, test_ind)

In [ ]:
'''
#print(test_label)
y = []
for k in test_label:
  y = y+k
yPred = []
for k in pred:
  for i in k:
    if i[0]>0.5:
      yPred.append(1)
    else :
      yPred.append(0)

print(len(y))
print(len(yPred))
'''


In [ ]:

pre = np.vstack(pred)

def getY(labels):
  y = []
  for k in labels:
    for i in k:
      yNew = 0
      if i == 1:
        yNew=1
      else:
        yNew=0 
      y.append(yNew)
  return(np.array(y))

def getPred(pred):
  y = []
  for k in pred:
    if k[0]>k[1]:
      y.append(0)
    else:
      y.append(1)
  return(np.array(y))

y = getY(test_label)
yPre = getPred(pre)

from sklearn.metrics import recall_score
rc = recall_score(y, yPred)
print('Recall is : ', rc)

In [ ]:
from sklearn.metrics import precision_score
ps = precision_score(y, yPred)
print('Precision score is : ', rc)